# Install Requirements
Let's get this out of the way up front!

**Note: Run with GPU or TPU environment!**

> Click `Runtime >> Change runtime type`

In [ ]:
!pip install wikipedia --quiet
!pip install spacy --quiet
!pip install pysbd --quiet
!pip install tensorflow-gpu==1.15.0 --quiet #--force-reinstall
!pip install gpt2-client==2.1.5 --quiet --no-dependencies #--force-reinstall 

# Download Wikipedia Articles
First, we need a corpus of relatively clean data. Wikipedia is crowd-sourced and written in modern English. Therefore we can trust that it is a good source of semantically, syntactically, and rhetorically sound text.

In [ ]:
import wikipedia

# todo: come up with a cool way to automatically create topic search terms
keywords = ['india', 'ocean', 'astronomy', 'economics', 'economy', 'earth', 
            'english', 'bacon', 'egg', 'dinosaur', 'rabbit', 'america', 'usa']

def save_article(title, article):
  with open('wiki_' + title + '.txt', 'w', encoding='utf-8') as outfile:
    outfile.write(article)

for keyword in keywords:
  try:
    search = wikipedia.search(keyword)
    for result in search:
      article = wikipedia.page(result)
      #print(result, article.url)
      save_article(result, article.content)
  except Exception as oops:
    #print(oops)
    continue
print('Done saving articles!')

# Parse Articles
The articles need to be split up into usable chunks. This uses regex to identify the section headers and split each article into single lines of text for each section. Furthermore, it looks at the number of word characters vs other characters to identify those sections that likely contain text instead of tables or other data.

In [ ]:
import os 
import re

result = list()

for file in os.listdir('.'):
  if not 'wiki_' in file:
    continue
  #print(file)
  with open(file, 'r', encoding='utf-8') as infile:
    text = infile.read()
  sections = re.split(r'={2,}.{0,80}={2,}', text)
  for section in sections:
    try:
      trimmed = section.strip()
      wordchars = re.findall(r'\w', trimmed)
      ratio = len(wordchars) / len(trimmed)
      if ratio > 0.80:
        final = re.sub(r'\s+', ' ', trimmed)
        result.append(final)
      # it seems like a ratio of greater than 80% word chars is ideal
    except:
      continue
  
print('Wikipedia sections parsed:', len(result))
with open('wikiparsed.txt', 'w', encoding='utf-8') as outfile:
  for line in result:
    outfile.write(line+'\n')

# Split Sentences
For the sake of simplicity, we don't want to go overboard and evaluate entire paragraphs. We want to only train on individual sentences. So let's use SpaCy and PYSBD (Python Sentence Boundary Detector) to split the corpus into sentences.

In [ ]:
import spacy
from pysbd.utils import PySBDFactory

nlp = spacy.blank('en')
nlp.add_pipe(PySBDFactory(nlp))
infile = 'wikiparsed.txt'
outfile = 'wikisentences.txt'
result = list()

with open('wikiparsed.txt', 'r', encoding='utf-8') as infile:
  lines = infile.readlines()
for line in lines:
  doc = nlp(line)
  #print('Parsing line:', line[0:80])
  for sent in list(doc.sents):
    result.append(sent)
    #print(sent)
#print('Sentences found:', len(result))
with open('wikisentences.txt', 'w', encoding='utf-8') as file:
  for line in result:
    if str(line) == '':
      continue
    file.write(str(line)+'\n')
print(outfile, 'saved!')

# Generate Gibberish v1
We have a great source of sentences that are semantically, syntactically, and rhetorically sound. The simplest way to generate gibberish, then, would be to scramble these sentences! For this first version, we want words, just all mixed up. This will create good training data because the samples will contain the same exact words as the sound sentences but out of order.

In [ ]:
from random import shuffle, seed

infile = 'wikisentences.txt'
outfile = 'wikiscrambled.txt'
result = list()

def scramble_sentence(sentence):
  sentence = sentence.strip()
  split = sentence.split()
  shuffle(split)
  return ' '.join(split)

seed()
with open(infile, 'r', encoding='utf-8') as file:
  lines = file.readlines()
for line in lines:
  line = line.strip()
  if line == '':
    continue
  scrambled = scramble_sentence(line)
  result.append(scrambled)
  #print('Scrambled sentence:', scrambled[0:100])
with open(outfile, 'w', encoding='utf-8') as file:
  for line in result:
    file.write(line+'\n')
print(outfile, 'saved!')        

# Generate Gibberish v2
This step may not be necessary but I'd like to be able to detect utter nonsense as well. So let's scramble all the characters in each sentence completely. I figure it's better to show the model random noise as well as random words.

In [ ]:
from random import shuffle, seed

infile = 'wikisentences.txt'
outfile = 'wikiscrambled2.txt'
result = list()

def scramble_sentence(sentence):
  sentence = sentence.strip()
  sentence = list(sentence)
  shuffle(sentence)
  return ''.join(sentence)

seed()
with open(infile, 'r', encoding='utf-8') as file:
  lines = file.readlines()
for line in lines:
  line = line.strip()
  if line == '':
    continue
  scrambled = scramble_sentence(line)
  result.append(scrambled)
  #print('Scrambled sentence:', scrambled[0:100])
with open(outfile, 'w', encoding='utf-8') as file:
  for line in result:
    file.write(line+'\n')
print(outfile, 'saved!')

# Compile Training Corpus
Let's build a training corpus that we can feed to GPT2! We need to bake the label directly into each line. Change `max_samples` to adjust corpus size. Multiple trainings may be necessary. Limits to finetuning memory requirements. I will add updates about limits and constraints as I figure them out. 

In [ ]:
from random import sample, seed

files = [
('wikisentences.txt', 'Clean'), 
('wikiscrambled2.txt', 'Gibberish'), 
('wikiscrambled.txt', 'Gibberish')
]

result = list()
max_samples = 100
corpus = 'corpus.txt' 

for file in files:
  with open(file[0], 'r', encoding='utf-8') as infile:
    lines = infile.readlines()
  for line in lines:
    line = line.strip()
    if line == '':
      continue
    line = '// %s || %s' % (line, file[1])
    result.append(line)
    #print(file, line[0:80])

seed()
subset = sample(result, max_samples)

with open(corpus, 'w', encoding='utf-8') as outfile:
  for line in subset:
    outfile.write(line+'\n\n')
print(corpus, 'saved!')

# Fine Tune GPT2!
This is where the rubber meets the road! Let's see if we can finetune a GPT-2 model! Obviously, the bigger the model, the better the results. But bigger models require more memory. There's a tradeoff between model size and corpus size. I will try to figure out the best results and update with recommended defaults here.

In [ ]:
from gpt2_client import GPT2Client

gpt2 = GPT2Client('345M')  # options: 117M, 345M, 774M, or 1558M
gpt2.load_model(force_download=False) 

corpus = 'corpus.txt'

result = gpt2.finetune(corpus, return_text=True)
print(result)

[7 | 654.66] loss=5.68 avg=5.09
[8 | 744.68] loss=5.34 avg=5.12
[9 | 836.34] loss=4.89 avg=5.09
